# 🎓 中文情感分析系统：交互式教学教程

## 👋 欢迎！
欢迎来到这份专为学习者设计的 **交互式 Jupyter Notebook** 教程。

**本项目的目标**：我们将从零开始，构建一个能够理解中文评论“情绪”的人工智能模型。不是简单地调用 API，而是亲手训练一个工业级的 **BERT** 模型。

## 📚 你将学到什么？
1.  **环境配置**：如何利用 Mac 的 MPS 加速深度学习。
2.  **数据工程**：从 Hugging Face 获取数据，并清洗、统一。
3.  **模型原理**：BERT 是如何理解中文的？
4.  **模型训练**：如何进行微调 (Fine-tuning) 以适应特定任务。
5.  **模型应用**：如何用自己训练的模型来分析一句话。

---

## 1️⃣ 第一步：导入工具包与环境检查

在开始做菜之前，我们需要先把锅碗瓢盆（工具包）准备好。

**核心工具介绍**：
*   **Transformers**: 由 Hugging Face 提供，是目前全世界最流行的 NLP 库，用来加载 BERT 模型。
*   **Datasets**:这也是 Hugging Face 的产品，用来下载与处理海量数据。
*   **Pandas**: 用来像 Excel 一样查看数据表格。
*   **Torch**: Pytorch 深度学习框架，我们的“引擎”。

In [ ]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# === 硬件加速检查 ===
# 深度学习需要大量的矩阵计算，CPU 算得太慢。
# Mac 电脑有专门的 MPS (Metal Performance Shaders) 加速芯片。
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("✅ 恭喜！检测到 Mac MPS 硬件加速，训练速度将起飞！🚀")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("✅ 检测到 NVIDIA CUDA，将使用 GPU 训练。")
else:
    device = torch.device("cpu")
    print("⚠️ 未检测到 GPU，将使用 CPU 训练。速度可能会比较慢，请耐心等待。☕️")

## 2️⃣ 第二步：配置参数 (Config)

为了让代码整洁，我们将所有的“设置项”都放在这里。这就好比做菜前的“菜谱”。

*   **BASE_MODEL**: 我们选用的基底模型是 `bert-base-chinese`，它是谷歌训练好的、已经读过几亿字中文的“高材生”。
*   **NUM_EPOCHS**: 训练轮数。设为 3，意味着模型会把我们的教材从头到尾看 3 遍。

In [ ]:
class Config:
    # 基模型：BERT 中文版
    BASE_MODEL = "google-bert/bert-base-chinese"
    
    # 分类数量：3类 (消极-0, 中性-1, 积极-2)
    NUM_LABELS = 3
    
    # 每一句话最长处理多少个字？超过的截断，不足的补0
    MAX_LENGTH = 128
    
    # 路径配置
    OUTPUT_DIR = "../checkpoints/tutorial_model"
    
    # 训练超参数
    BATCH_SIZE = 16  # 一次可以并行处理多少句话 (看显存大小)
    LEARNING_RATE = 2e-5  # 学习率：模型学得太快容易学偏，太慢容易学不会。2e-5 是经验值。
    NUM_EPOCHS = 3   # 训练几轮
    
    # 标签字典
    ID2LABEL = {0: 'Negative (消极)', 1: 'Neutral (中性)', 2: 'Positive (积极)'}
    LABEL2ID = {'negative': 0, 'neutral': 1, 'positive': 2}

print("配置加载完毕。")

## 3️⃣ 第三步：准备数据 (Data Preparation)

我们的策略是 **“混合双打”**：
1.  **通用数据** (`clapAI`): 包含日常生活的各种评论，让模型懂常识。
2.  **垂直数据** (`OpenModels`): 包含中医药领域的评论，让模型懂行话。

下面的代码会自动从网络加载这些数据，并进行清洗。

In [ ]:
# 加载 Tokenizer (分词器)
# 它的作用是把汉字转换成模型能读懂的数字 ID
tokenizer = AutoTokenizer.from_pretrained(Config.BASE_MODEL)

def prepare_dataset():
    print("⏳ 正在加载数据 (可能需要一点时间下载)...")
    
    # 为了演示速度，我们只取前 1000 条数据 (正式训练时会用全部数据)
    # 如果电脑性能好，可以把 split="train[:1000]" 改成 split="train"
    sample_size = 500
    
    # 1. 加载通用情感数据
    ds_clap = load_dataset("clapAI/MultiLingualSentiment", split=f"train[:{sample_size}]", trust_remote_code=True)
    ds_clap = ds_clap.filter(lambda x: x['language'] == 'zh') # 只留中文
    
    # 2. 加载中医药情感数据
    ds_med = load_dataset("OpenModels/Chinese-Herbal-Medicine-Sentiment", split=f"train[:{sample_size}]", trust_remote_code=True)
    
    # 3. 统一列名
    # 不同数据集的列名可能不一样，我们要把它们统一改成 'text' 和 'label'
    if 'review_text' in ds_med.column_names: ds_med = ds_med.rename_column('review_text', 'text')
    if 'sentiment_label' in ds_med.column_names: ds_med = ds_med.rename_column('sentiment_label', 'label')
    
    # 4. 合并数据集
    common_cols = ['text', 'label']
    combined = concatenate_datasets([ds_clap.select_columns(common_cols), ds_med.select_columns(common_cols)])
    
    # 5. 数据清洗与统一标签
    def process_data(example):
        # 统一标签为数字 0, 1, 2
        lbl = example['label']
        if isinstance(lbl, str):
            lbl = lbl.lower()
            if lbl in ['negative', '0']: lbl = 0
            elif lbl in ['neutral', '1']: lbl = 1
            elif lbl in ['positive', '2']: lbl = 2
        return {'labels': int(lbl)}
        
    combined = combined.map(process_data)
    
    # 6. 分词 (Tokenization)
    def tokenize(batch):
        return tokenizer(batch['text'], padding="max_length", truncation=True, max_length=Config.MAX_LENGTH)
        
    print("✂️ 正在进行分词处理...")
    tokenized_ds = combined.map(tokenize, batched=True)
    
    # 7. 划分训练集和验证集 (90% 训练, 10% 验证)
    return tokenized_ds.train_test_split(test_size=0.1)

# 执行数据准备
dataset = prepare_dataset()
print(f"\n✅ 数据准备完成！\n训练集大小: {len(dataset['train'])} 条\n测试集大小: {len(dataset['test'])} 条")

## 4️⃣ 第四步：数据可视化 (Data Visualization)

很多时候模型训练不好是因为数据分布不均匀（比如全是好评，那模型只要一直猜好评准确率也很高，但这没用）。
让我们画个饼图来看看我们的数据怎么样。

In [ ]:
# 从 dataset 中提取 label 列
train_labels = dataset['train']['labels']

# 统计每个类别的数量
labels_count = pd.Series(train_labels).value_counts().sort_index()
labels_name = [Config.ID2LABEL[i] for i in labels_count.index]

# 由于 Matplotlib 默认不支持中文，我们用英文显示或者设置字体，这里为了简单直接用英文
plt.figure(figsize=(8, 5))
plt.pie(labels_count, labels=labels_name, autopct='%1.1f%%', colors=['#ff9999','#66b3ff','#99ff99'])
plt.title('Training Data Distribution')
plt.show()

## 5️⃣ 第五步：模型训练 (Model Training)

这是最激动人心的一步！我们将启动 Hugging Face `Trainer`。

我们将实现一个**“智能跳过”**逻辑：如果检测到之前已经训练好了模型，就直接加载，不再浪费时间重新训练。

In [ ]:
# 定义评价指标：我们需要知道模型的准确率(Accuracy)
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc}

# 检查是否已存在
if os.path.exists(Config.OUTPUT_DIR) and os.path.exists(os.path.join(Config.OUTPUT_DIR, "config.json")):
    print(f"🎉 检测到已训练的模型: {Config.OUTPUT_DIR}")
    print("🚀 直接加载模型，跳过训练！")
    model = AutoModelForSequenceClassification.from_pretrained(Config.OUTPUT_DIR)
    model.to(device)
else:
    print("💪 未找到已训练模型，开始新一轮训练...")
    
    # 加载初始模型
    model = AutoModelForSequenceClassification.from_pretrained(Config.BASE_MODEL, num_labels=Config.NUM_LABELS)
    model.to(device)
    
    # 设置训练参数
    training_args = TrainingArguments(
        output_dir=Config.OUTPUT_DIR,
        num_train_epochs=Config.NUM_EPOCHS,
        per_device_train_batch_size=Config.BATCH_SIZE,
        evaluation_strategy="epoch", # 每个 Epoch 结束后评估一次
        save_strategy="epoch",       # 每个 Epoch 结束后保存一次
        logging_steps=10,
        report_to="none"             # 不上报到wandb
    )
    
    # 初始化训练器
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset['train'],
        eval_dataset=dataset['test'],
        processing_class=tokenizer,
        compute_metrics=compute_metrics
    )
    
    # 开始训练！
    trainer.train()
    
    # 保存最终结果
    trainer.save_model(Config.OUTPUT_DIR)
    tokenizer.save_pretrained(Config.OUTPUT_DIR)
    print("💾 训练完成，模型已保存！")

## 6️⃣ 第六步：互动测试 (Inference Demo)

现在模型已经“毕业”了，让我们来考考它！
在下面的输入框里随便输入一句话（支持中文），点击“分析”看看它觉得的情感是什么。

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# 预测函数
def predict_sentiment(text):
    # 1. 预处理
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # 2. 模型推理
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        
    # 3. 结果解析
    pred_idx = torch.argmax(probs).item()
    confidence = probs[0][pred_idx].item()
    label = Config.ID2LABEL[pred_idx]
    
    return label, confidence

# 界面组件
text_box = widgets.Text(placeholder='请输入要分析的句子...', description='评论:', layout=widgets.Layout(width='400px'))
btn_run = widgets.Button(description="开始分析", button_style='primary')
output_area = widgets.Output()

def on_click(b):
    with output_area:
        output_area.clear_output()
        text = text_box.value
        if not text:
            print("❌ 请先输入内容！")
            return
        
        print(f"🔍 正在分析: \"{text}\"")
        label, conf = predict_sentiment(text)
        
        # 只有置信度高才显示绿色，否则显示黄色
        icon = "✅" if conf > 0.8 else "🤔"
        print(f"{icon} 预测结果: [{label}] ")
        print(f"📊 置信度: {conf*100:.2f}%")

btn_run.on_click(on_click)
display(text_box, btn_run, output_area)